In [1]:
'''In this notebook we will download all of the IBL video files and then compress them to 64x? pixels and 30fps
  then we will save them to a local folder from which we can run an http server'''
import numpy as np
import requests
import subprocess

In [2]:
from one.api import ONE
one = ONE(base_url='https://alyx.internationalbrainlab.org')

Downloading: C:\Users\Dan\Downloads\ONE\alyx.internationalbrainlab.org\tmpj_iznyhi\cache.zip Bytes: 77519661


100%|████████████████████████████████████████████████████| 73.92850971221924/73.92850971221924 [00:17<00:00,  4.31it/s]


In [3]:
eid = '0802ced5-33a3-405e-8336-b65ebc5cb07c'
print(eid)

0802ced5-33a3-405e-8336-b65ebc5cb07c


In [4]:
with open('username.txt') as f:
    username = f.readlines()
    username = username[0]
with open('password.txt') as f:
    password = f.readlines()
    password = password[0]


In [7]:
dsets = one.type2datasets(eid, '_iblrig_Camera.raw', details=True)
videos = ['left','right','body']

# opener.addheaders[]

for video in videos:
  dset = next(d for idx, d in dsets.iterrows() if video in d['rel_path'])
  url = one.record2url(dset)
  print(url)
  ftext = eid + '_' + video
  videoFile = 'videos/' +ftext + ".mp4"
  open(videoFile, 'wb').write(requests.get(url, auth=(username, password)).content)
  # also get the timestamps
  # To get timestamps for video data so you can find out what times each frame is at
  dsets_time = one.type2datasets(eid, 'camera.times')
  dset = next(d for d in dsets_time if video in d)
  # load data
  times = one.load_dataset(eid, dset)
  print(len(times))
  print(1/np.diff(times))
  with open('C:\\proj\\VBL\\iblreplay-preproc\\videos\\'+ftext+'_times.txt','w') as f:
    f.write(str(times[0]))
  np.save('videos/'+ftext+"_times.npy",times)
  call = subprocess.call(['ffmpeg',
                 '-i', 'C:\\proj\\VBL\\iblreplay-preproc\\videos\\'+ftext+'.mp4',
                 '-vf', 'fps=24,scale=64:52', 
                 'C:\\proj\\VBL\\iblreplay-preproc\\videos\\'+ftext+'_scaled'+'.mp4'])

https://ibl.flatironinstitute.org/mainenlab/Subjects/ZFM-02373/2021-06-23/001/raw_video_data/_iblrig_leftCamera.raw.757e3f76-d9c9-4cfa-b0d8-6a78e6e54140.mp4
231420
[60.12625311 60.12625311 60.0060006  ... 60.12625311 60.0060006
 60.12625311]
https://ibl.flatironinstitute.org/mainenlab/Subjects/ZFM-02373/2021-06-23/001/raw_video_data/_iblrig_rightCamera.raw.12fe2ea5-0796-4004-a730-10d52c1ec150.mp4
579247
[ 37.64491882 150.0150015  150.76884573 ... 150.01500151 150.76884572
 150.01500151]
https://ibl.flatironinstitute.org/mainenlab/Subjects/ZFM-02373/2021-06-23/001/raw_video_data/_iblrig_bodyCamera.raw.d56a60b1-e183-4708-9483-1bb7040bf7b8.mp4
116108
[30.15376915 30.12349428 30.15376915 ... 30.15376915 30.15376915
 30.15376915]


In [8]:
times

array([  42.96763614,   43.00079949,   43.03399617, ..., 3894.08735232,
       3894.12051567, 3894.15367902])